In [1]:
import os

import analysis_session_helper_functions_c3 as helper
import analysis_utils as utils

import pandas as pd

In [2]:
cohort = 'cohort_3'
to_analyze = 'full_clean'
data_dir = '/Users/rebekahzhang/Documents/shuler_lab/behavior_data'
data_folder = os.path.join(data_dir, cohort, to_analyze)
stitched_folder = os.path.join(data_dir, cohort, 'full_clean_stitched')
stitched_all_mice_folder = os.path.join(data_dir, cohort, 'full_clean_stitched_all_mice')
print(data_folder)

/Users/rebekahzhang/Documents/shuler_lab/behavior_data/cohort_3/full_clean


# Generates all session logs
does not need to be run if session logs have been generated

In [3]:
session_log = helper.generate_all_session_log(data_folder)
session_log.head(10)

,date,mouse,dir,filename
298,2023-07-22,RZ019,2023-07-22_12-39-24_RZ019,data_RZ019_2023-07-22_12-39-24.txt
25,2023-07-22,RZ020,2023-07-22_12-16-16_RZ020,data_RZ020_2023-07-22_12-16-16.txt
179,2023-07-22,RZ021,2023-07-22_12-47-56_RZ021,data_RZ021_2023-07-22_12-47-56.txt
104,2023-07-23,RZ018,2023-07-23_13-12-32_RZ018,data_RZ018_2023-07-23_13-12-32.txt
32,2023-07-23,RZ019,2023-07-23_13-38-21_RZ019,data_RZ019_2023-07-23_13-38-21.txt
110,2023-07-23,RZ020,2023-07-23_13-16-23_RZ020,data_RZ020_2023-07-23_13-16-23.txt
83,2023-07-23,RZ021,2023-07-23_13-51-08_RZ021,data_RZ021_2023-07-23_13-51-08.txt
53,2023-07-24,RZ018,2023-07-24_10-25-02_RZ018,data_RZ018_2023-07-24_10-25-02.txt
85,2023-07-24,RZ019,2023-07-24_10-49-20_RZ019,data_RZ019_2023-07-24_10-49-20.txt
169,2023-07-24,RZ020,2023-07-24_10-28-11_RZ020,data_RZ020_2023-07-24_10-28-11.txt


get a list of mouse names

In [4]:
# prints mouse names in data folder, check for weird ones and delete from data base
mouse_list = utils.generate_mouse_list(session_log)
print(mouse_list)

['RZ018', 'RZ019', 'RZ020', 'RZ021', 'RZ022', 'RZ023', 'RZ024', 'RZ025']


get the type of training for session meta data and add to log

In [5]:
training_list = []
rig_list = []
for dir_name, file_name in zip(session_log.dir, session_log.filename):
    try:
        session_meta = utils.load_session_meta(data_folder, dir_name, file_name)
        training = session_meta.training.tolist()[0]
        rig = session_meta.rig.tolist()[0]
        training_list.append(training)
        rig_list.append(rig)
    except:
        print(dir_name)

session_log['training'] = training_list
session_log['rig'] = rig_list

add columns of basic info to each session to log

In [6]:
session_basics_columns = ['num_blocks', 'num_trials', 'rewards', 'time', 'proper_end']
for dir_name, file_name in zip(session_log.dir, session_log.filename):
    try:
        session = utils.load_session(data_folder, dir_name, file_name)
        session_basic = helper.get_session_basics(session)
        session_log.loc[session_log.dir == dir_name, session_basics_columns] = session_basic
    except:
        print(dir_name)

### Focus only on regular training sessions

In [7]:
training_session_log = session_log.loc[session_log.training == 'regular'].sort_values('dir').reset_index()
training_session_log.head()

,index,date,mouse,dir,filename,training,rig,num_blocks,num_trials,rewards,time,proper_end
0,201,2023-07-25,RZ018,2023-07-25_10-24-14_RZ018,data_RZ018_2023-07-25_10-24-14.txt,regular,rig2,1.0,353.0,181.5,1767.68,True
1,269,2023-07-25,RZ020,2023-07-25_10-28-09_RZ020,data_RZ020_2023-07-25_10-28-09.txt,regular,rig3,1.0,363.0,137.9,2528.20,True
2,143,2023-07-25,RZ019,2023-07-25_10-56-16_RZ019,data_RZ019_2023-07-25_10-56-16.txt,regular,rig2,1.0,340.0,97.2,1639.49,True
3,285,2023-07-25,RZ021,2023-07-25_11-12-50_RZ021,data_RZ021_2023-07-25_11-12-50.txt,regular,rig3,1.0,355.0,629.6,2835.79,True
4,190,2023-07-25,RZ018,2023-07-25_13-05-38_RZ018,data_RZ018_2023-07-25_13-05-38.txt,regular,rig3,1.0,338.0,303.5,1915.50,True


### Examine quality of sessions
doesn't need to run when data folder is cleaned

check for short sessions, prints out dir

In [ ]:
short_session = training_session_log.loc[(training_session_log['training'] == 'regular') & 
                                         (training_session_log['num_trials'] < 50)] 
print(short_session.dir, short_session.num_trials)

check for missing sessions by the number of sessions in each training day

In [ ]:
num_mice = 8
date_list = training_session_log.date.unique().tolist()
for date in date_list:
    data = training_session_log.loc[training_session_log['date'] == date]
    if len(data) < num_mice:
        print(date)

check for same mouse with multiple sessions per day, prints out date and mouse if too many

In [ ]:
# prints dates of the same mouse with multiple sessions
for d in training_session_log.date.unique().tolist():
    session_of_the_day = training_session_log.loc[training_session_log['date'] == d]
    for mouse in mouse_list:
        count = len(session_of_the_day.loc[session_of_the_day['mouse'] == mouse])
        if count > 1:
            print(d, mouse)   

make a copy of cleaned data before preceeding!

### Add training session number to training log

In [8]:
training_session_log.sort_values(by=['mouse', 'dir'], inplace=True)
for mouse in mouse_list:
    total_sessions = sum(training_session_log.mouse == mouse)
    training_session_log.loc[training_session_log.mouse == mouse, 'session_num'] = list(range(total_sessions))
training_session_log.session_num = training_session_log.session_num.astype(int)

In [9]:
training_session_log.session_num.max()

38

### Saves all sessions log and training session log

In [10]:
filename = f'all_sessions.csv'
path = os.path.join(data_folder, filename)
session_log.to_csv(path)

In [11]:
filename = f'training_sessions.csv'
path = os.path.join(data_folder, filename)
training_session_log.to_csv(path)

# Generate all trials per session

load session log

In [12]:
training_session_log = utils.load_session_log(data_folder, 'training_sessions.csv')

### Raw data processing and generate initial all trials df
adds trial numbers and states to hardware entries
create all trials df with trial basics added
both files saved, 
does't need to be rerun

In [13]:
all_trials_column_names = ['session_trial_num', 'block_trial_num', 'block_num', 'start_time', 'end_time']

In [14]:
for dir_name, file_name in zip(training_session_log.dir, training_session_log.filename):
    
    processed_path = os.path.join(data_folder, dir_name, f'processed_{file_name[:-4]}.csv')
    all_trials_path =os.path.join(data_folder, dir_name, f'{dir_name}_all_trials.csv')
    if os.path.isfile(processed_path) and os.path.isfile(all_trials_path):
        continue

    session = utils.load_session(data_folder, dir_name, file_name)
    session.session_time = session.session_time - session.iloc[0]['session_time']
    session['trial_time'] = ''
    total_trial_list = helper.generate_total_trial_list(training_session_log, dir_name)
    all_trials = pd.DataFrame(columns=all_trials_column_names)

    for t in total_trial_list:
        trial = session.loc[session['session_trial_num'] == t]
        
        trial_basics = helper.get_trial_basics(trial)
        helper.align_trial_number(session, trial_basics)

        trial_state_times = helper.get_trial_state_times(trial)
        helper.align_trial_states(session, trial_state_times, trial_basics)

        trial = session.loc[session['session_trial_num'] == t]
        helper.add_trial_time(session, t, trial, trial_basics)

        trial_basics = pd.DataFrame([trial_basics])
        all_trials = pd.concat([all_trials, trial_basics], ignore_index=True)
    
    session = utils.trim_session(training_session_log, dir_name, session)
    session.to_csv(processed_path)
    all_trials.to_csv(all_trials_path)

### Adding analyzed trial data to all trials df

In [15]:
all_trials_data_column = ['bg_length', 'blk_type', 'num_bg_lick',
                          'miss_trial', 'time_waited', 'reward', 'num_consumption_lick',
                          'good_trial']

In [16]:
for dir_name, file_name in zip(training_session_log.dir, training_session_log.filename):
    all_trials_analyzed_path =os.path.join(data_folder, dir_name, f'{dir_name}_all_trials_analyzed.csv')
    # if os.path.isfile(all_trials_analyzed_path):
    #     continue

    processed_session = utils.load_processed_session(data_folder, dir_name, file_name)
    all_trials = utils.load_all_trials(data_folder, dir_name)
    total_trial_list = helper.generate_total_trial_list(training_session_log, dir_name)

    for t in total_trial_list:
        trial = processed_session.loc[processed_session['session_trial_num'] == t]
        trial_performance = helper.get_trial_performance(trial)
        all_trials.loc[all_trials['session_trial_num'] == t, all_trials_data_column] = trial_performance
    
    all_trials.to_csv(all_trials_analyzed_path)

# Combine sessions

## Stitch all sessions from the same day

load session log and generate lists for looping

In [17]:
training_session_log = utils.load_session_log(data_folder, 'training_sessions.csv')
unique_date_list = utils.generate_date_list(training_session_log)
unique_mouse_list = utils.generate_mouse_list(training_session_log)

makes the stitched session log with columns of date, mouse, dir, filename

In [18]:
stitched_session_log = helper.generate_stitched_session_log(training_session_log)
stitched_session_log.head(10)

,date,mouse,dir,filename,num_sessions
0,2023-07-25,RZ018,2023-07-25_RZ018,processed_data_RZ018_2023-07-25.csv,3
1,2023-07-25,RZ019,2023-07-25_RZ019,processed_data_RZ019_2023-07-25.csv,3
2,2023-07-25,RZ020,2023-07-25_RZ020,processed_data_RZ020_2023-07-25.csv,2
3,2023-07-25,RZ021,2023-07-25_RZ021,processed_data_RZ021_2023-07-25.csv,2
4,2023-07-26,RZ018,2023-07-26_RZ018,processed_data_RZ018_2023-07-26.csv,1
5,2023-07-26,RZ019,2023-07-26_RZ019,processed_data_RZ019_2023-07-26.csv,1
6,2023-07-26,RZ020,2023-07-26_RZ020,processed_data_RZ020_2023-07-26.csv,1
7,2023-07-26,RZ021,2023-07-26_RZ021,processed_data_RZ021_2023-07-26.csv,1
8,2023-07-26,RZ022,2023-07-26_RZ022,processed_data_RZ022_2023-07-26.csv,1
9,2023-07-26,RZ023,2023-07-26_RZ023,processed_data_RZ023_2023-07-26.csv,1


### Stitch!

creates empty directories in the stitched folder

In [19]:
for d in stitched_session_log.dir:
    new_dir = os.path.join(stitched_folder, d)
    if not os.path.exists(new_dir):
        os.makedirs(new_dir)

loop through each day and each mouse to stitch processed_session and all_trials_analyzed

In [20]:
for d in unique_date_list:
    log_date = training_session_log.loc[training_session_log.date == d]
    for m in unique_mouse_list:
        log_date_mouse = log_date.loc[log_date.mouse == m]
        num_sessions = len(log_date_mouse)

        if num_sessions == 0:
            continue
        
        else:
            stitched_processed_session = utils.load_processed_session(data_folder, log_date_mouse.dir.iloc[0], log_date_mouse.filename.iloc[0])
            stitched_all_trials = utils.load_all_trials_analyzed(data_folder, log_date_mouse.dir.iloc[0])

            for i in range(1, num_sessions):
                session_2 = utils.load_processed_session(data_folder, log_date_mouse.dir.iloc[i], log_date_mouse.filename.iloc[i])
                all_trials_2 = utils.load_all_trials_analyzed(data_folder, log_date_mouse.dir.iloc[i])
                
                stitched_processed_session = helper.stitch_sessions(stitched_processed_session, session_2)
                stitched_all_trials = helper.stitch_all_trials(stitched_all_trials, all_trials_2)

            stitched_processed_session.to_csv(utils.generate_stitched_session_path(stitched_folder, m, d))
            stitched_all_trials.to_csv(utils.generate_stitched_all_trials_path(stitched_folder, m, d))

### Add info to stitched sesscion log and save it

In [21]:
session_basics_columns = ['num_blocks', 'num_trials', 'rewards', 'time', 'proper_end']
for m, d in zip(stitched_session_log.mouse, stitched_session_log.date):
    session = utils.load_stitched_session(stitched_folder, m, d)
    session_basic = helper.get_session_basics(session)
    stitched_session_log.loc[stitched_session_log.dir == f'{d}_{m}', session_basics_columns] = session_basic

In [22]:
stitched_session_log.sort_values(by=['mouse', 'date'], inplace=True)
for mouse in mouse_list:
    total_sessions = sum(stitched_session_log.mouse == mouse)
    stitched_session_log.loc[stitched_session_log.mouse == mouse, 'session_num'] = list(range(total_sessions))
stitched_session_log.session_num = stitched_session_log.session_num.astype(int)

In [23]:
filename = f'stitched_training_sessions.csv'
path = os.path.join(stitched_folder, filename)
stitched_session_log.to_csv(path)

## Stitch all Sessions from the same day from all mice

load stitched session log

In [24]:
stitched_session_log = utils.load_session_log(stitched_folder, 'stitched_training_sessions.csv')

In [25]:
unique_date_list = utils.generate_date_list(stitched_session_log)
unique_mouse_list = utils.generate_mouse_list(stitched_session_log)

generage stitched all mice log with columns of date, mouse, dir, filename

In [26]:
stitched_all_mice_session_log = helper.generate_stitched_all_mice_session_log(stitched_session_log)

In [27]:
for date in stitched_session_log.date:
    new_dir = os.path.join(stitched_all_mice_folder, date)
    if not os.path.exists(new_dir):
        os.makedirs(new_dir)

In [28]:
for d in unique_date_list:
    log_date = stitched_session_log.loc[stitched_session_log.date == d]
    num_mice = len(log_date)

    if num_mice == 0:
        continue
    else:
        stitched_processed_session =  utils.load_stitched_session(stitched_folder, log_date.mouse.iloc[0], log_date.date.iloc[0])
        stitched_processed_session['mouse'] = log_date.mouse.iloc[0]
        stitched_all_trials = utils.load_stitched_all_trials_analyzed(stitched_folder, log_date.mouse.iloc[0], log_date.date.iloc[0])
        stitched_all_trials['mouse'] = log_date.mouse.iloc[0]

        for i in range(1, num_mice):
            session_2 = utils.load_stitched_session(stitched_folder, log_date.mouse.iloc[i], log_date.date.iloc[i])
            session_2['mouse'] = log_date.mouse.iloc[i]
            all_trials_2 = utils.load_stitched_all_trials_analyzed(stitched_folder, log_date.mouse.iloc[i], log_date.date.iloc[i])
            all_trials_2['mouse'] = log_date.mouse.iloc[i]
            
            stitched_processed_session = helper.stitch_sessions(stitched_processed_session, session_2)
            stitched_all_trials = helper.stitch_all_trials(stitched_all_trials, all_trials_2)
        
        stitched_processed_session.to_csv(utils.generate_stitched_all_mice_session_path(stitched_all_mice_folder, d))
        stitched_all_trials.to_csv(utils.generate_stitched_all_mice_all_trials_path(stitched_all_mice_folder, d))

In [29]:
session_basics_columns = ['num_blocks', 'num_trials', 'rewards', 'time', 'proper_end']
for d in stitched_all_mice_session_log.date:
    session = utils.load_stitched_all_mice_session(stitched_all_mice_folder, d)
    session_basic = helper.get_session_basics(session)
    stitched_all_mice_session_log.loc[stitched_all_mice_session_log.date == d, session_basics_columns] = session_basic

In [30]:
total_days = len(stitched_all_mice_session_log)
stitched_all_mice_session_log['days'] = list(range(total_days))

In [31]:
filename = f'stitched_all_mice_training_sessions.csv'
path = os.path.join(stitched_all_mice_folder, filename)
stitched_all_mice_session_log.to_csv(path)